In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time

In [99]:
# 加载你的数据文件 (文件名请替换成你自己的)
df_hpi = pd.read_csv(r'C:\Users\Playtech\Desktop\Ass3\Ass3 new\hm10modify.csv')
df_ocr = pd.read_csv(r'C:\Users\Playtech\Desktop\Ass3\Ass3 new\OCRmodify.csv')
df_cpi = pd.read_csv(r'C:\Users\Playtech\Desktop\Ass3\Ass3 new\consumers-price-index-june-2025-quarter-seasonally-adjusted.csv')
df_gdp = pd.read_csv(r'C:\Users\Playtech\Desktop\Ass3\Ass3 new\gross-domestic-product-june-2025-quarter.csv')

In [101]:
# 设置图表风格
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [103]:
# --- HPI ---
# 查看前几行
print("HPI Data Head:")
print(df_hpi.head())

# 查看数据信息，检查数据类型和缺失值
print("\nHPI Data Info:")
df_hpi.info()

HPI Data Head:
  Unnamed: 0      House sales House price index (HPI)  \
0        NaN         (Number)                 (Index)   
1      Notes              NaN                     NaN   
2       Unit               no                   Index   
3  Series Id  QVB.Q.MR0H01.na          HPI.Q.H01T0.ia   
4    3月 1990           22,855                     477   

  Total value of housing stock Residential investment (GDP)  
0                         ($m)                    (Real $m)  
1                          Yes                          NaN  
2                         NZDm                      NZDm(r)  
3                     HHAL.QC1                GDE.Q.EI24.RA  
4                      123,000                        1,790  

HPI Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                   

In [105]:
# 1. 真实的数据是从第5行（索引为4）开始的，我们先把它选出来
df_hpi_clean = df_hpi.iloc[4:].copy()
# 2. 重置索引，让新数据从0开始
df_hpi_clean.reset_index(drop=True, inplace=True)
# 3. 选择我们需要的列，并给它们起一个简洁的新名字
#   注意: 'Unnamed: 0' 是日期那一列的默认名字
df_hpi_clean = df_hpi_clean[['Unnamed: 0', 'House price index (HPI)']]
df_hpi_clean.columns = ['Date', 'HPI']
df_hpi_clean['Date'] = df_hpi_clean['Date'].str.replace('月 ', '/', regex=False)
df_hpi_clean['Date'] = pd.to_datetime(df_hpi_clean['Date'], format='%m/%Y')
#   将HPI列转换为数字（浮点数）
df_hpi_clean['HPI'] = df_hpi_clean['HPI'].astype(float)

In [107]:
print(df_hpi_clean.head())
df_hpi_clean.info()

        Date    HPI
0 1990-03-01  477.0
1 1990-06-01  480.0
2 1990-09-01  483.0
3 1990-12-01  482.0
4 1991-03-01  479.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    142 non-null    datetime64[ns]
 1   HPI     141 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.3 KB


In [109]:
#--- 检查ocr ---
print(df_ocr.head())

         Date   Ocr
0   2000/3/31  5.31
1   2000/6/30  6.21
2   2000/9/30  6.50
3  2000/12/31  6.50
4   2001/3/31  6.45


In [111]:
# 1. 将 'Date' 列安全地转换为标准的datetime对象
#    Pandas通常能自动识别 '2000/3/31' 这种格式
df_ocr['Date'] = pd.to_datetime(df_ocr['Date'])
# 2. 将所有日期都标准化为当月的第一天
#    .dt.to_period('M') 会将日期截断到月份 (例如 '2000-03')
#    .dt.to_timestamp() 会再将这个月份转换回日期，并默认设置为该月的第一天
df_ocr['Date'] = df_ocr['Date'].dt.to_period('M').dt.to_timestamp()


In [113]:
#--- 再次检查 ocr ----
print(df_ocr.head())
df_ocr.info()

        Date   Ocr
0 2000-03-01  5.31
1 2000-06-01  6.21
2 2000-09-01  6.50
3 2000-12-01  6.50
4 2001-03-01  6.45
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    115 non-null    datetime64[ns]
 1   Ocr     115 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.9 KB


In [115]:
print(df_cpi.head())

  Series_reference   Period  Data_value   STATUS  UNITS Subject  \
0      CPIQ.SE9S01  2006.06       762.0  REVISED  Index     CPI   
1      CPIQ.SE9S01  2006.09       771.0  REVISED  Index     CPI   
2      CPIQ.SE9S01  2006.12       780.0  REVISED  Index     CPI   
3      CPIQ.SE9S01  2007.03       788.0  REVISED  Index     CPI   
4      CPIQ.SE9S01  2007.06       794.0  REVISED  Index     CPI   

                                               Group       Series_title_1  \
0  CPI Level 1 Groups for New Zealand, Seasonally...  Seasonally adjusted   
1  CPI Level 1 Groups for New Zealand, Seasonally...  Seasonally adjusted   
2  CPI Level 1 Groups for New Zealand, Seasonally...  Seasonally adjusted   
3  CPI Level 1 Groups for New Zealand, Seasonally...  Seasonally adjusted   
4  CPI Level 1 Groups for New Zealand, Seasonally...  Seasonally adjusted   

  Series_title_2  
0           Food  
1           Food  
2           Food  
3           Food  
4           Food  


In [127]:
df_cpi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11785 entries, 0 to 11784
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Series_reference  11785 non-null  object 
 1   Period            11785 non-null  object 
 2   Data_value        11781 non-null  float64
 3   STATUS            11785 non-null  object 
 4   UNITS             11785 non-null  object 
 5   Subject           11785 non-null  object 
 6   Group             11785 non-null  object 
 7   Series_title_1    11785 non-null  object 
 8   Series_title_2    11785 non-null  object 
dtypes: float64(1), object(8)
memory usage: 828.8+ KB


In [117]:
# 这会打印出该列所有不重复的值
#CPIQ.SE9SA
print(df_cpi['Series_reference'].unique())

['CPIQ.SE9S01' 'CPIQ.SE9S011' 'CPIQ.SE9S01101' 'CPIQ.SE9S01102'
 'CPIQ.SE9S012' 'CPIQ.SE9S01201' 'CPIQ.SE9S01202' 'CPIQ.SE9S013'
 'CPIQ.SE9S01301' 'CPIQ.SE9S01302' 'CPIQ.SE9S01303' 'CPIQ.SE9S01304'
 'CPIQ.SE9S01305' 'CPIQ.SE9S01306' 'CPIQ.SE9S014' 'CPIQ.SE9S01401'
 'CPIQ.SE9S01402' 'CPIQ.SE9S015' 'CPIQ.SE9S01501' 'CPIQ.SE9S01502'
 'CPIQ.SE9S02' 'CPIQ.SE9S021' 'CPIQ.SE9S02101' 'CPIQ.SE9S02102'
 'CPIQ.SE9S02103' 'CPIQ.SE9S022' 'CPIQ.SE9S02200' 'CPIQ.SE9S03'
 'CPIQ.SE9S031' 'CPIQ.SE9S03102' 'CPIQ.SE9S03103' 'CPIQ.SE9S03104'
 'CPIQ.SE9S03105' 'CPIQ.SE9S03106' 'CPIQ.SE9S03107' 'CPIQ.SE9S032'
 'CPIQ.SE9S03202' 'CPIQ.SE9S03203' 'CPIQ.SE9S03204' 'CPIQ.SE9S04'
 'CPIQ.SE9S041' 'CPIQ.SE9S04101' 'CPIQ.SE9S042' 'CPIQ.SE9S04201'
 'CPIQ.SE9S043' 'CPIQ.SE9S04301' 'CPIQ.SE9S04302' 'CPIQ.SE9S044'
 'CPIQ.SE9S04401' 'CPIQ.SE9S04402' 'CPIQ.SE9S04403' 'CPIQ.SE9S04404'
 'CPIQ.SE9S045' 'CPIQ.SE9S04501' 'CPIQ.SE9S04502' 'CPIQ.SE9S04503'
 'CPIQ.SE9S05' 'CPIQ.SE9S051' 'CPIQ.SE9S05101' 'CPIQ.SE9S05102'
 'CPIQ.SE9